In [2]:
# Verbindung zur SQLite-Datenbank
import sqlite3
import os

if os.path.exists("lab13.db"):
    os.remove("lab13.db")

conn = sqlite3.connect("lab13.db")
cursor = conn.cursor()

In [3]:
# Studenten-Tabelle (Matrikelnummer als PRIMARY KEY)
cursor.execute('''
CREATE TABLE studenten (
    matrikelnummer TEXT PRIMARY KEY,
    name TEXT NOT NULL
)
''')

# Dozenten-Tabelle
cursor.execute('''
CREATE TABLE dozenten (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL
)
''')

# Fächer-Tabelle
cursor.execute('''
CREATE TABLE faecher (
    kuerzel TEXT PRIMARY KEY,
    bezeichnung TEXT NOT NULL,
    wochenstunden INTEGER,
    dozent_id INTEGER,
    FOREIGN KEY (dozent_id) REFERENCES dozenten(id)
)
''')

# Prüfungen-Tabelle mit Note, Gewichtung und Bezeichnung
cursor.execute('''
CREATE TABLE pruefungen (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    student_id TEXT,
    fach_kuerzel TEXT,
    note REAL,
    gewichtung INTEGER CHECK(gewichtung IN (1, 2)),
    bezeichnung TEXT,
    FOREIGN KEY (student_id) REFERENCES studenten(matrikelnummer),
    FOREIGN KEY (fach_kuerzel) REFERENCES faecher(kuerzel)
)
''')


In [4]:
# Studenten anlegen
studenten = [
    ('20230001', 'Alice'),
    ('20230002', 'Bob'),
    ('20230003', 'Charlie'),
    ('20230004', 'Diana'),
    ('20230005', 'Eve')

]
cursor.executemany("INSERT INTO studenten (matrikelnummer, name) VALUES (?, ?)", studenten)

# Dozenten einfügen
dozenten = ['Dr. Maier', 'Prof. Schmitt', 'Dr. Klein', 'Prof. Wagner']
for name in dozenten:
    cursor.execute("INSERT INTO dozenten (name) VALUES (?)", (name,))

# Dozenten-IDs holen
cursor.execute("SELECT id FROM dozenten WHERE name = 'Dr. Maier'")
maier_id = cursor.fetchone()[0]
cursor.execute("SELECT id FROM dozenten WHERE name = 'Prof. Schmitt'")
schmitt_id = cursor.fetchone()[0]
cursor.execute("SELECT id FROM dozenten WHERE name = 'Dr. Klein'")
klein_id = cursor.fetchone()[0]
cursor.execute("SELECT id FROM dozenten WHERE name = 'Prof. Wagner'")
wagner_id = cursor.fetchone()[0]

# Fächer einfügen
faecher = [
    ('MATH', 'Mathematik', 4, maier_id),
    ('INFO', 'Informatik', 3, schmitt_id),
    ('PHYS', 'Physik', 4, klein_id),
    ('ELEK', 'Elektronik', 3, wagner_id),
    ('STAT', 'Statistik', 2, maier_id),
]
cursor.executemany("INSERT INTO faecher VALUES (?, ?, ?, ?)", faecher)


In [5]:
# Tabelle zurücksetzen (falls Zelle mehrfach ausgeführt wird)
cursor.execute("DELETE FROM pruefungen")

# Daten anlegen
pruefungen = [
    ('20230001', 'MATH', 1.7, 2, 'Klausur'),
    ('20230001', 'INFO', 1.3, 2, 'Projektarbeit'),
    ('20230001', 'PHYS', 2.0, 1, 'Mündliche Prüfung'),
    ('20230001', 'PHYS', 3.0, 1, 'Kurztest'),

    ('20230002', 'MATH', 2.3, 2, 'Klausur'),
    ('20230002', 'INFO', 1.0, 2, 'Projektarbeit'),
    ('20230002', 'PHYS', 2.7, 1, 'Mündliche Prüfung'),
    ('20230002', 'ELEK', 1.5, 1, 'Laborarbeit')
]
cursor.executemany(
    "INSERT INTO pruefungen (student_id, fach_kuerzel, note, gewichtung, bezeichnung) VALUES (?, ?, ?, ?, ?)",
    pruefungen
)

## SQL-Abfragen

### Alle Prüfungen mit Matrikelnummer, Fach, Art, Note, Gewichtung

In [6]:
cursor.execute('''
SELECT p.student_id, p.fach_kuerzel, p.bezeichnung, p.note, p.gewichtung
FROM pruefungen p
''')
for row in cursor.fetchall():
    print(row)

('20230001', 'MATH', 'Klausur', 1.7, 2)
('20230001', 'INFO', 'Projektarbeit', 1.3, 2)
('20230001', 'PHYS', 'Mündliche Prüfung', 2.0, 1)
('20230001', 'PHYS', 'Kurztest', 3.0, 1)
('20230002', 'MATH', 'Klausur', 2.3, 2)
('20230002', 'INFO', 'Projektarbeit', 1.0, 2)
('20230002', 'PHYS', 'Mündliche Prüfung', 2.7, 1)
('20230002', 'ELEK', 'Laborarbeit', 1.5, 1)


### Alle Prüfungen mit Studierendenname, Fach, Art, Note, Gewichtung
(JOIN verwenden)

In [7]:
cursor.execute('''
SELECT s.matrikelnummer, s.name, f.bezeichnung, p.note, p.gewichtung
FROM studenten s
JOIN pruefungen p ON s.matrikelnummer = p.student_id
JOIN faecher f ON p.fach_kuerzel = f.kuerzel
WHERE p.note IS NOT NULL
''')
for row in cursor.fetchall():
    print(row)


('20230001', 'Alice', 'Mathematik', 1.7, 2)
('20230001', 'Alice', 'Informatik', 1.3, 2)
('20230001', 'Alice', 'Physik', 2.0, 1)
('20230001', 'Alice', 'Physik', 3.0, 1)
('20230002', 'Bob', 'Mathematik', 2.3, 2)
('20230002', 'Bob', 'Informatik', 1.0, 2)
('20230002', 'Bob', 'Physik', 2.7, 1)
('20230002', 'Bob', 'Elektronik', 1.5, 1)


### Durchschnitt aller Noten pro Student (ungewichtet)
(ROUND und GROUP BY verwenden)

In [12]:
cursor.execute('''
SELECT s.matrikelnummer, s.name, f.bezeichnung, p.note, p.gewichtung
FROM studenten s
ROUND JOIN pruefungen p ON s.matrikelnummer = p.student_id
GROUP BY s.matrikelnummer, f.bezeichnung, p.note, p.gewichtung
HAVING COUNT(p.id) > 0
''')
for row in cursor.fetchall():
    print(row)


OperationalError: near "ROUND": syntax error

### Gewichtete Gesamtnote pro Student und Fach

In [9]:
cursor.execute('''
SELECT s.matrikelnummer, s.name, f.bezeichnung, p.note, p.gewichtung
FROM studenten s
JOIN pruefungen p ON s.matrikelnummer = p.student_id
JOIN faecher f ON p.fach_kuerzel = f.kuerzel
WHERE p.note IS NOT NULL
''')

for row in cursor.fetchall():
    print(row)

('20230001', 'Alice', 'Mathematik', 1.7, 2)
('20230001', 'Alice', 'Informatik', 1.3, 2)
('20230001', 'Alice', 'Physik', 2.0, 1)
('20230001', 'Alice', 'Physik', 3.0, 1)
('20230002', 'Bob', 'Mathematik', 2.3, 2)
('20230002', 'Bob', 'Informatik', 1.0, 2)
('20230002', 'Bob', 'Physik', 2.7, 1)
('20230002', 'Bob', 'Elektronik', 1.5, 1)


### Anzahl der Prüfungen pro Fach
(COUNT(*) verwenden)

In [10]:
cursor.execute('''
SELECT s.matrikelnummer, s.name, f.bezeichnung, p.note, p.gewichtung
FROM studenten s
JOIN pruefungen p ON s.matrikelnummer = p.student_id
JOIN faecher f ON p.fach_kuerzel = f.kuerzel
WHERE p.note IS NOT NULL
''')
for row in cursor.fetchall():
    print(row)

('20230001', 'Alice', 'Mathematik', 1.7, 2)
('20230001', 'Alice', 'Informatik', 1.3, 2)
('20230001', 'Alice', 'Physik', 2.0, 1)
('20230001', 'Alice', 'Physik', 3.0, 1)
('20230002', 'Bob', 'Mathematik', 2.3, 2)
('20230002', 'Bob', 'Informatik', 1.0, 2)
('20230002', 'Bob', 'Physik', 2.7, 1)
('20230002', 'Bob', 'Elektronik', 1.5, 1)


### Alle Prüfungen der Art „Klausur“

In [1]:
cursor.execute('''
SELECT p.student_id, p.fach_kuerzel, p.bezeichnung, p.note, p.gewichtung
FROM pruefungen p
JOIN pruefungen p ON s.matrikelnummer = p.student_id
JOIN faecher f ON p.fach_kuerzel = f.kuerzel
WHERE p.note IS NOT NULL
''')
for row in cursor.fetchall():
    print(row)

NameError: name 'cursor' is not defined

### Alle Fächer von Dr. Maier

In [ ]:
cursor.execute('''
SELECT s.matrikelnummer, s.name, f.bezeichnung, p.note, p.gewichtung
FROM studenten s
JOIN pruefungen p ON s.matrikelnummer = p.student_id
JOIN faecher f ON p.fach_kuerzel = f.kuerzel 
WHERE p.note IS NOT NULL
''')
for row in cursor.fetchall():
    print(row)